<h3>Векторизуем словарь Meddra</h3>

In [1]:
from vectorization import ConceptVectorizer

In [2]:
CV = ConceptVectorizer('cointegrated/rubert-tiny2', '../../Data/External/pt_rus.asc')

loading model...


Some weights of the model checkpoint at cointegrated/rubert-tiny2 were not used when initializing BertModel: ['cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.bias', 'cls.predictions.decoder.bias', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.weight', 'cls.predictions.decoder.weight', 'cls.predictions.transform.dense.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [3]:
CV.fit_transform()

getting concept embeddings...


In [4]:
CV.thesaurus_embeddings

tensor([[-0.5571,  0.4180,  0.3808,  ...,  0.5321,  0.8140, -1.6517],
        [-0.0364, -0.3752,  0.2873,  ...,  0.4556,  0.6893, -0.5252],
        [-0.1648, -0.4934,  0.5567,  ...,  0.6926,  0.2847, -0.4295],
        ...,
        [-0.0708,  0.0196,  0.7664,  ..., -0.2933,  0.9758, -1.3564],
        [-0.2319, -0.6193, -0.0307,  ...,  0.4852,  0.1158, -0.8784],
        [ 0.0246, -0.9299,  0.0626,  ...,  0.2106,  0.2250, -0.9511]])

In [5]:
print(len(CV.meddra_codes))

23954


<h3>Создадим датасет RDR</h3>

In [6]:
import jsonlines
import numpy as np
from sklearn.model_selection import train_test_split

In [7]:
ds = []
with jsonlines.open('../../Data/Raw/medNorm_14012022.jsonlines') as reader:
    for obj in reader:
        ds.append(obj)

In [8]:
X_train, X_test = train_test_split(ds, test_size=0.33, random_state=42)

In [9]:
#выцепим фразы с нормализацией по Meddra без их контекста

train_phrases = []
train_concepts = []

test_phrases = []
test_concepts = []

#markup_errors = 0

for review in X_train:
    for ent in review['objects']['MedEntity']:
        if 'MedDRA' in ent.keys() and ent['MedDRA']!='':
            #try:
            train_concepts.append(CV.meddra_term_to_meddra_code[ent['MedDRA'].split('|')[0]])
            #except:
            #markup_errors+=1
            #continue
            train_phrases.append(ent['text'])
            
            
for review in X_test:
    for ent in review['objects']['MedEntity']:
        if 'MedDRA' in ent.keys() and ent['MedDRA']!='':
            #try:
            test_concepts.append(CV.meddra_term_to_meddra_code[ent['MedDRA'].split('|')[0]])
            #except:
            #    markup_errors+=1
            #    continue
            test_phrases.append(ent['text'])
            
#print('Число не найденных в словаре концептов: %s'%markup_errors)
print('Всего фраз в трейне: %s'%len(train_phrases))
print('Всего фраз в тесте: %s'%len(test_phrases))

print('Уникальных фраз в трейне: %s'%len(set(train_phrases)))
print('Уникальных фраз в тесте: %s'%len(set(test_phrases)))

#Посмотрим на статистику разбиения
print('%s концептов не входящих либо в трейн, либо в тест'%len(set.union(set(train_concepts), set(test_concepts)) - set.intersection(set(test_concepts), set(train_concepts))))
print('%s концептов, которые есть в тесте, но нет в трейне'%len(set(test_concepts) - set(train_concepts)))
print('%s концептов, которые есть в трейне, но нет в тесте'%len(set(train_concepts) - set(test_concepts)))

Всего фраз в трейне: 3476
Всего фраз в тесте: 1750
Уникальных фраз в трейне: 1204
Уникальных фраз в тесте: 750
162 концептов не входящих либо в трейн, либо в тест
53 концептов, которые есть в тесте, но нет в трейне
109 концептов, которые есть в трейне, но нет в тесте


In [10]:
#векторизуем для обучения и теста
from copy import copy
from transformers import AutoTokenizer
import torch

train_y = []
test_y = []

empty_vec = [0]*len(CV.meddra_codes)
for concept in train_concepts:
    y = copy(empty_vec)
    y[CV.meddra_codes.index(concept)] = 1
    train_y.append(y)
    
for concept in test_concepts:
    y = copy(empty_vec)
    y[CV.meddra_codes.index(concept)] = 1
    test_y.append(y)

train_y = torch.tensor(train_y)
test_y = torch.tensor(test_y)    

In [11]:
tokenizer = AutoTokenizer.from_pretrained('cointegrated/rubert-tiny2')
train_X = tokenizer(train_phrases, padding=True, truncation=True, return_tensors='pt')
test_X = tokenizer(test_phrases, padding=True, truncation=True, return_tensors='pt')

Проверка векторизации

In [12]:
from random import randint

i = randint(1, train_y.size()[0])
#phrase = train_X[i]
concept = train_y[i]

meddra_code = CV.meddra_codes[concept.argmax()]

print(train_phrases[i])
print(CV.meddra_code_to_meddra_term[meddra_code])

отек тканей
Отек кожи


Демонстрация того, как отрабатывает косинусная близость

In [22]:
import torch.nn.functional as F

a = CV.thesaurus_embeddings[0].unsqueeze(0).repeat(32, 1)
b = CV.thesaurus_embeddings#.unsqueeze(0).repeat(32, 1, 1)

a_n, b_n = a.norm(dim=1)[:, None], b.norm(dim=1)[:, None]
a_norm = a / torch.clamp(a_n, min=1e-6)
b_norm = b / torch.clamp(b_n, min=1e-6)
sim_mt = torch.mm(a_norm, b_norm.transpose(0, 1))
F.softmax(sim_mt, dim=1)

tensor([[6.7345e-05, 4.3767e-05, 4.5936e-05,  ..., 5.2762e-05, 4.4660e-05,
         4.3657e-05],
        [6.7345e-05, 4.3767e-05, 4.5936e-05,  ..., 5.2762e-05, 4.4660e-05,
         4.3657e-05],
        [6.7345e-05, 4.3767e-05, 4.5936e-05,  ..., 5.2762e-05, 4.4660e-05,
         4.3657e-05],
        ...,
        [6.7345e-05, 4.3767e-05, 4.5936e-05,  ..., 5.2762e-05, 4.4660e-05,
         4.3657e-05],
        [6.7345e-05, 4.3767e-05, 4.5936e-05,  ..., 5.2762e-05, 4.4660e-05,
         4.3657e-05],
        [6.7345e-05, 4.3767e-05, 4.5936e-05,  ..., 5.2762e-05, 4.4660e-05,
         4.3657e-05]])

<h2>Сама модель</h2>

In [13]:
import torch.nn as nn
import torch.nn.functional as F
from transformers import AutoModel, AutoConfig
from transformers import BertPreTrainedModel, BertModel

model = AutoModel.from_pretrained('cointegrated/rubert-tiny2')
cos = nn.CosineSimilarity(dim=1, eps=1e-9)
cfg = AutoConfig.from_pretrained('cointegrated/rubert-tiny2')

#encoded_input = tokenizer(sentences, padding=True, truncation=True, return_tensors='pt')

# Compute token embeddings
#with torch.no_grad():
#    model_output = model(**encoded_input)

def mean_pooling(model_output, attention_mask):
    token_embeddings = model_output[0] #First element of model_output contains all token embeddings
    input_mask_expanded = attention_mask.unsqueeze(-1).expand(token_embeddings.size()).float()
    return torch.sum(token_embeddings * input_mask_expanded, 1) / torch.clamp(input_mask_expanded.sum(1), min=1e-9)

normalized_embs = CV.thesaurus_embeddings.norm(dim=1)[:, None]
normalized_embs = CV.thesaurus_embeddings / torch.clamp(normalized_embs, min=1e-9)
normalized_embs = normalized_embs.transpose(0, 1)

class Net(BertPreTrainedModel):
    def __init__(self):
        #super().__init__()
        super(Net, self).__init__(cfg)
        self.emb = BertModel(cfg)

    def forward(self, x):
        emb = self.emb(**x)
        #with torch.no_grad():
        x = mean_pooling(emb, x['attention_mask'])
        #имеем две матрицы x - (batch_size, emb_size) и thesaurus_embeddings - (thesaurus_size, emb_size)
        #надо посчитать косинусную близость близость между каждым вектором x и каждым вложением из тезауруса
        #решение: https://stackoverflow.com/questions/50411191/how-to-compute-the-cosine-similarity-in-pytorch-for-all-rows-in-a-matrix-with-re
        x_n = x.norm(dim=1)[:, None] 
        x_n = x / torch.clamp(x_n, min=1e-9)
        #b_norm = CV.thesaurus_embeddings / torch.clamp(b_, min=1e-9)
        cos_sim = torch.mm(x_n, normalized_embs)
        x = F.softmax(cos_sim, dim=1)
        return x


net = Net()

Some weights of the model checkpoint at cointegrated/rubert-tiny2 were not used when initializing BertModel: ['cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.bias', 'cls.predictions.decoder.bias', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.weight', 'cls.predictions.decoder.weight', 'cls.predictions.transform.dense.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


<h2>Обучение модели</h2>

In [14]:
import torch.optim as optim
import torch

criterion = nn.CrossEntropyLoss()
optimizer = optim.AdamW(net.parameters())

In [15]:
class MedNormDataset(torch.utils.data.Dataset):
    def __init__(self, X, y):
        self.X = X
        if y.dtype==torch.int64:
            self.y = y.to(torch.float32)
        else:
            self.y = y
        
    def __len__(self):
        return self.X['input_ids'].size()[0]

    def __getitem__(self, idx):
        #if torch.is_tensor(idx):
        #    idx = idx.tolist()
        sample = {'tokenized_phrases': {'input_ids': self.X['input_ids'][idx], 
                                        'token_type_ids': self.X['token_type_ids'][idx],
                                        'attention_mask': self.X['attention_mask'][idx]},
                  'label': self.y[idx]}
        return sample

In [16]:
from tqdm import trange
from tqdm import tqdm


batch_size=2
RDR_ds = MedNormDataset(X=train_X, y=train_y)
trainloader = torch.utils.data.DataLoader(RDR_ds, batch_size=batch_size,
                                          shuffle=True)

net.train()
for epoch in range(1, 2):
    with tqdm(trainloader, unit="batch") as tepoch:
        for data in tepoch:

            tepoch.set_description(f"Epoch {epoch}")

            inputs = data['tokenized_phrases']
            labels = data['label']

            optimizer.zero_grad()    
            # forward + backward + optimize
            outputs = net(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
            
            
            #чтобы посчитать accuracy нужно конвертнуть в one-hot
            max_idx = torch.argmax(outputs, 1, keepdim=True)
            one_hot = torch.FloatTensor(outputs.shape)
            one_hot.zero_()
            one_hot.scatter_(1, max_idx, 1)
            correct = torch.all(torch.eq(labels, one_hot),  dim=1).sum().item()

            #correct = (one_hot == labels)
            accuracy = correct / batch_size
            tepoch.set_postfix(loss=loss.item(), accuracy=100. * accuracy)
            
             #optimizer.zero_grad()
             #output = model(data)
             #predictions = output.argmax(dim=1, keepdim=True).squeeze()
             #loss = F.nll_loss(output, target)
             #correct = (predictions == target).sum().item()
             #accuracy = correct / batch_size

             #loss.backward()
             #optimizer.step()

             #tepoch.set_postfix(loss=loss.item(), accuracy=100. * accuracy)
             #sleep(0.1)

#phbar = trange(trainloader, desc='Batch loss 0')
#f = open('log.txt', 'w')
#for epoch in range(10):  # loop over the dataset multiple times

    #running_loss = 0.0
    #for data in phbar:#enumerate((trainloader, 0), desc='Batch loss 0' ) 
        # get the inputs; data is a list of [inputs, labels]
        #inputs = data['tokenized_phrases']
        #labels = data['label']
        #print(inputs['input_ids'][0])
        # zero the parameter gradients
        #optimizer.zero_grad()    
        # forward + backward + optimize
        #outputs = net(inputs)
        #loss = criterion(outputs, labels)
        #f.write('%s\n'%loss.item())
        #phbar.set_description('Batch loss %s'%loss.item())
        #loss.backward()
        #optimizer.step()
        # print statistics
        #running_loss += loss.item()
        #if i % 2000 == 1999:    # print every 2000 mini-batches
        #    print(f'[{epoch + 1}, {i + 1:5d}] loss: {running_loss / 2000:.3f}')
        #    running_loss = 0.0
#f.close()
print('Finished Training')

Epoch 1: 100%|██████████| 1738/1738 [03:41<00:00,  7.85batch/s, accuracy=50, loss=10.1] 

Finished Training


<h2>Тест</h2>

In [18]:
from sklearn.metrics import classification_report, f1_score

net.eval()

model_answers=[]
real_answers=[]

for i, phrase in enumerate(test_phrases):
    #i = randint(1, train_y.size()[0])
    encoded_phrase = {'input_ids': test_X['input_ids'][i].unsqueeze(0), 
              'token_type_ids': test_X['token_type_ids'][i].unsqueeze(0),
              'attention_mask': test_X['attention_mask'][i].unsqueeze(0)}

    concept = test_y[i]

    with torch.no_grad():
        meddra_code = CV.meddra_codes[net(encoded_phrase).argmax()]


    model_answers.append(meddra_code)
    #print('phrase: %s'%phrase)
    #print('model: %s'%CV.meddra_code_to_meddra_term[meddra_code])



    meddra_code = CV.meddra_codes[concept.argmax()]
    real_answers.append(meddra_code)
    #print(train_phrases[i])
    #print('real: %s'%CV.meddra_code_to_meddra_term[meddra_code])
    
#print(classification_report(real_answers, model_answers, zero_division=0))
f1_score(real_answers, model_answers, average='micro')

0.66

<h2>Инференс</h2>

In [21]:
from random import randint

i = randint(1, train_y.size()[0])

net.eval()
phrase = {'input_ids': train_X['input_ids'][i].unsqueeze(0), 
          'token_type_ids': train_X['token_type_ids'][i].unsqueeze(0),
          'attention_mask': train_X['attention_mask'][i].unsqueeze(0)}

concept = train_y[i]

with torch.no_grad():
    meddra_code = CV.meddra_codes[net(phrase).argmax()]
    


print('phrase: %s'%train_phrases[i])
print('model: %s'%CV.meddra_code_to_meddra_term[meddra_code])



meddra_code = CV.meddra_codes[concept.argmax()]

#print(train_phrases[i])
print('real: %s'%CV.meddra_code_to_meddra_term[meddra_code])

phrase: температура 40
model: Повышенная температура тела
real: Пирексия


<h2>Сохранение и загрузка модели</h2>

In [22]:
!mkdir ./cadec_SoTa_on_RDR_rubert_tiny/

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


In [24]:
torch.save(net, './cadec_SoTa_on_RDR_rubert_tiny.pt')

In [25]:
the_model = torch.load('./cadec_SoTa_on_RDR_rubert_tiny.pt')

Покажем, что это ТА ЖЕ модель

In [26]:
from sklearn.metrics import classification_report, f1_score

the_model.eval()

model_answers=[]
real_answers=[]

for i, phrase in enumerate(test_phrases):
    #i = randint(1, train_y.size()[0])
    encoded_phrase = {'input_ids': test_X['input_ids'][i].unsqueeze(0), 
              'token_type_ids': test_X['token_type_ids'][i].unsqueeze(0),
              'attention_mask': test_X['attention_mask'][i].unsqueeze(0)}

    concept = test_y[i]

    with torch.no_grad():
        meddra_code = CV.meddra_codes[the_model(encoded_phrase).argmax()]


    model_answers.append(meddra_code)
    #print('phrase: %s'%phrase)
    #print('model: %s'%CV.meddra_code_to_meddra_term[meddra_code])



    meddra_code = CV.meddra_codes[concept.argmax()]
    real_answers.append(meddra_code)
    #print(train_phrases[i])
    #print('real: %s'%CV.meddra_code_to_meddra_term[meddra_code])
    
#print(classification_report(real_answers, model_answers, zero_division=0))
f1_score(real_answers, model_answers, average='micro')

0.66